In [1]:
import cv2
import numpy as np
import subprocess
from tqdm import tqdm
import os

In [2]:
# Define the function to segment cracks
def segment_cracks(image_path, output_path):
    command = f"python predict.py --config configs/hrsegnetb32.yml --model_path hrsegnetb32/best_model/model.pdparams --image_path {image_path} --save_dir {output_path}"
    subprocess.run(command, shell=True)

In [11]:
import os 

os.chdir("../")

In [12]:
import cv2

# Attempt to read the image
input_image = cv2.imread("018_11.jpg")

# Check if the image was read successfully
if input_image is None:
    print("Error: Unable to read the image file.")
else:
    # Get the dimensions of the image
    height, width, _ = input_image.shape
    print("Image dimensions - Height:", height, "Width:", width)


Image dimensions - Height: 500 Width: 500


In [13]:
total_segmented_area = 0

In [15]:
import os

# Define the parent folder path
parent_folder = "hrseg/results"

# Create parent folder and subfolders if they don't exist
os.makedirs(parent_folder, exist_ok=True)
os.makedirs(os.path.join(parent_folder, "gray_scale"), exist_ok=True)
os.makedirs(os.path.join(parent_folder, "after_dilated"), exist_ok=True)


In [16]:
crop_filename = "018_11.jpg"

In [17]:
segment_cracks(crop_filename, 'output/result')

In [18]:
hsv_image = cv2.cvtColor(cv2.imread(crop_filename), cv2.COLOR_BGR2HSV)


In [19]:

lower_green = np.array([35, 50, 50])  # Lower bound for green color
upper_green = np.array([90, 255, 255])  # Upper bound for green color

In [20]:
mask = cv2.inRange(hsv_image, lower_green, upper_green)

In [21]:
segmented_image = np.where(mask == 255, 1, 0).astype(np.uint8) * 255

In [23]:
output_dir = "hrseg/results/gray_scale"
cv2.imwrite(f"{output_dir}/wall_image.jpg", segmented_image)

True

In [24]:
kernel = np.ones((250, 250), np.uint8)

In [26]:
dilated_image = cv2.dilate(segmented_image, kernel, iterations=1)
output_dir1 = "hrseg/results/after_dilated"
cv2.imwrite(f"{output_dir1}/wall_image.jpg", dilated_image)
print(dilated_image)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
segmented_area = np.count_nonzero(dilated_image) / (height * width) * 100
total_segmented_area += segmented_area
# Calculate the overall percentage of damage of the wall
overall_percentage_damage = total_segmented_area

print(f"Overall percentage of damage: {overall_percentage_damage:.2f}%")

Overall percentage of damage: 0.00%
